# Query Cell Types Ontology

# Context

This notebook has been put together for the MMB demo on 2022-05-30: see [slides](https://docs.google.com/presentation/d/1Ib1_8byK0hVuNS-wPbqmeL5Lcf67m_oegC3-czPw5ws/edit#slide=id.g116ba5ed71e_0_8)

## Imports

In [ ]:
import json
import rdflib
import getpass
import pandas as pd
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
from bmo_tools.ontologies import subontology_from_term
import pprint
from kgforge.core import KnowledgeGraphForge

## Setup

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN,
                            endpoint="https://staging.nise.bbp.epfl.ch/nexus/v1",
                            bucket="neurosciencegraph/datamodels")

## Ontologies

### Brain Region Ontology  from WebProtégé

In [ ]:
brain_region_ontology = rdflib.Graph()
brain_region_ontology.parse("./brainregion.ttl") # Provide the filepath of the Brain Region Ontology donwloaded from WebProtégé

## Queries

### Which brain regions do have neuron t-types available

In [ ]:
query = f"""

PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX nsg: <https://neuroshapes.org/>
PREFIX mty: <https://bbp.epfl.ch/ontologies/core/mtypes/>

SELECT ?brain_region

WHERE {{
        ?t_type_id rdfs:subClassOf ?region_restriction ;
                  rdfs:label ?t_type ;
                  rdfs:subClassOf* <https://bbp.epfl.ch/ontologies/core/celltypes/NeuronTranscriptomicType> .
        ?region_restriction a owl:Restriction ;
           owl:onProperty bmo:canHaveBrainRegion ;
           owl:someValuesFrom ?brain_region_id .        
        ?brain_region_id rdfs:label ?brain_region .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=1000, rewrite=False)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
set(df.brain_region)

### Get possible t-types for a given brain region

In [ ]:
query = f"""

PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX nsg: <https://neuroshapes.org/>
PREFIX mty: <https://bbp.epfl.ch/ontologies/core/mtypes/>

SELECT ?brain_region ?t_type

WHERE {{
        ?t_type_id rdfs:subClassOf ?region_restriction ;
                  rdfs:label ?t_type .
        ?region_restriction a owl:Restriction ;
           owl:onProperty bmo:canHaveBrainRegion ;
           owl:someValuesFrom <{brain_region}> .        
        <{brain_region}> rdfs:label ?brain_region . 
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100, rewrite=False)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head()

### Get possible met-type combinations plus excitatory/inhibitory category for a given brain region

Set brain region label

In [ ]:
BRAIN_REGION = "Cerebral cortex"
# BRAIN_REGION = "Cerebellum"

Get brain region id

In [ ]:
brain_region = [s for s, p, o in brain_region_ontology.triples((None, RDFS.label, rdflib.term.Literal(BRAIN_REGION)))][0]

In [ ]:
brain_region

In [ ]:
query = f"""

PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX nsg: <https://neuroshapes.org/>
PREFIX mty: <https://bbp.epfl.ch/ontologies/core/mtypes/>

SELECT ?brain_region ?transmitter ?t_type ?m_type ?e_type

WHERE {{
        ?t_type_id rdfs:subClassOf ?region_restriction ;
                  rdfs:label ?t_type .
        ?region_restriction a owl:Restriction ;
           owl:onProperty bmo:canHaveBrainRegion ;
           owl:someValuesFrom <{brain_region}> .        
        <{brain_region}> rdfs:label ?brain_region .
        
        ?m_type_id rdfs:subClassOf ?t_restriction ;
            rdfs:label ?m_type ;
            rdfs:subClassOf* nsg:MType .
        ?t_restriction a owl:Restriction ;
           owl:onProperty bmo:canHaveTType ;
           owl:someValuesFrom ?t_type_id .

        ?m_type_id rdfs:subClassOf+ ?transmitter_restriction .
        ?transmitter_restriction a owl:Restriction ;
            owl:onProperty mty:hasNeurotransmitterType ;
            owl:someValuesFrom ?transmitter_id .
        ?transmitter_id rdfs:label ?transmitter . 

        ?e_type_id rdfs:subClassOf ?m_restriction ;
            rdfs:subClassOf ?t_restriction_2 ;
            rdfs:label ?e_type ;
            rdfs:subClassOf* nsg:EType .
        ?m_restriction a owl:Restriction ;
           owl:onProperty bmo:canHaveMType ;
           owl:someValuesFrom ?m_type_id .
        ?t_restriction_2 a owl:Restriction ;
           owl:onProperty bmo:canHaveTType ;
           owl:someValuesFrom ?t_type_id .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100, rewrite=False)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head()

### Get m-types with a basket cell morphology

In [ ]:
query = f"""

PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX nsg: <https://neuroshapes.org/>
PREFIX mty: <https://bbp.epfl.ch/ontologies/core/mtypes/>

SELECT ?basket_cell

WHERE {{
        ?basket_cell_id rdfs:subClassOf+ / nsg:hasMorphologicalPhenotype / rdfs:label "basket cell morphology" ;
                  rdfs:label ?basket_cell .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100, rewrite=False)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head(20)

### Get definition of a basket cell morphology

In [ ]:
query = f"""

PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX nsg: <https://neuroshapes.org/>
PREFIX mty: <https://bbp.epfl.ch/ontologies/core/mtypes/>

SELECT ?basket_cell ?definition

WHERE {{
        ?basket_cell_id rdfs:subClassOf+ / nsg:hasMorphologicalPhenotype ?pato_id ;
            rdfs:label ?basket_cell .
        ?pato_id rdfs:label "basket cell morphology" ;
                <http://purl.obolibrary.org/obo/IAO_0000115> ?definition .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=20000, rewrite=False)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
set(df.definition)

### Get t-types from Yao et al. 2021 paper

In [ ]:
query = f"""

PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX nsg: <https://neuroshapes.org/>
PREFIX mty: <https://bbp.epfl.ch/ontologies/core/mtypes/>

SELECT ?label

WHERE {{
        ?id rdfs:seeAlso <https://www.sciencedirect.com/science/article/pii/S0092867421005018?dgcid=rss_sd_all> ;
            rdfs:label ?label .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100, rewrite=False)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df